In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path 

from tests import *

from plotting import plot_qdc, plot_tdc, plot_testpulse, plot_disc_calibration


In [5]:
test_map = {"Aldo": Aldo,
            "QDCCalibration": QDCCalibration,
            "TDCCalibration": TDCCalibration,
            "TestPulse": TestPulse,
            "ExtTestPulse": ExtTestPulse,
            "DiscCalibration0": DiscCalibration_0,
            "DiscCalibration1": DiscCalibration_1,
            "DiscCalibration2": DiscCalibration_2,
            "DiscCalibration3": DiscCalibration_3,
            "QDCCalibration0": QDCCalibration_0,
            "QDCCalibration1": QDCCalibration_1,
            "QDCCalibration2": QDCCalibration_2,
            "QDCCalibration3": QDCCalibration_3,
            "QDCCalibration4": QDCCalibration_4,
            "QDCCalibration5": QDCCalibration_5,
            "QDCCalibration6": QDCCalibration_6,
            "QDCCalibration7": QDCCalibration_7,
            "TDCCalibration": TDCCalibration}


      
class TestResult:

    def __init__(self,
                path: str | Path = "",
                tests: list[str] = ["Aldo",
                                    "TestPulse",
                                    "ExtTestPulse",
                                    "DiscCalibration0",
                                    "DiscCalibration1",
                                    "DiscCalibration2",
                                    "DiscCalibration3",
                                    "QDCCalibration0",
                                    "QDCCalibration1",
                                    "QDCCalibration2",
                                    "QDCCalibration3",
                                    "QDCCalibration4",
                                    "QDCCalibration5",
                                    "QDCCalibration6",
                                    "QDCCalibration7",
                                    "TDCCalibration"],
                ) -> None:
      
        self.path = path
        self.tests = tests

        # for each test assert that the corresponding file exists
        for test in self.tests:
            datafile = test_map[test](test_result_dir=self.path).datafile
            if not datafile.exists(): 
                raise FileNotFoundError(f"File {datafile} not found in {self.path}")

        self.serial_files = [f for f in Path(self.path).rglob('SN_3*.txt')]
        assert len(self.serial_files) > 0, "No serial files found in the directory"

        self.tester_to_serial = {}
        for f in self.serial_files: 
            sn = int(f.stem.split(" ")[-1])
            with open(f, "r") as file:
                tester = int(file.read().strip())
            self.tester_to_serial[tester] = sn


    def get_data(self,
                test: str,
                filename: str = ""
                ) -> pd.DataFrame:
        if test not in self.tests:
            raise ValueError(f"Test {test} not in {self.tests}")
        
        test_args = {"test_result_dir": self.path,
                    "tester_to_serial": self.tester_to_serial}
        if filename != "":
            test_args["filename"] = filename
            return test_map[test](**test_args).get_data()


In [3]:
#skip_dirs = ["/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319",
             #"/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320",
             #"/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407081902"] 
# get all test result dirs
test_result_dirs = [d for d in Path("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data").glob("2024*")
                    if d.is_dir()] #and str(d) not in skip_dirs]
# assert that the dirs are sorted by their timestamps (yyyymmddhhmm)
test_result_dirs = sorted(test_result_dirs, key=lambda x: int(x.stem))
test_result_dirs

[PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041556'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041910'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042048'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042204'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407050847'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051014'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051151'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051322'),
 PosixPath

In [4]:
trs = []
for d in test_result_dirs:
    try:
        tr = TestResult(d)
        trs.append(tr)
    except FileNotFoundError as e:
        print(e)
        continue

File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319/aldo.tsv not found
File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320/aldo.tsv not found
File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407081902/fetp_tres_scan.tsv not found


In [5]:
aldo = Aldo(test_result_dir=str(test_result_dirs[0]), tester_to_serial=trs[0].tester_to_serial)
aldo_data = aldo.get_data()
aldo_data

,tester_ID,asic_id,side,gain,DAC,Vout,current,SN
0,0,0,0,0,0,35.377575,0.000067,334
1,0,0,1,0,0,35.377575,0.000066,334
2,0,1,0,0,0,34.709000,0.000066,334
3,0,1,1,0,0,34.467175,0.000066,334
4,1,0,0,0,0,35.036175,0.000066,290
...,...,...,...,...,...,...,...,...
16379,6,1,1,1,255,42.362050,0.000037,318
16380,7,0,0,1,255,41.878400,0.000038,229
16381,7,0,1,1,255,41.835725,0.000037,229
16382,7,1,0,1,255,42.290925,0.000038,229


In [6]:
reduced_aldo = aldo.get_reduced_data()

/eos/home-j/jowulff/MTD/combined_plots/tests.py:91: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


In [7]:
reduced_aldo

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN
0,0,0,0,0,0.020495,35.412772,2.191727,334.0
1,0,0,0,1,0.040751,32.309174,4.052352,334.0
2,0,0,1,0,0.020592,35.413578,2.137131,334.0
3,0,0,1,1,0.041010,32.312781,4.770937,334.0
4,0,1,0,0,0.020424,34.736938,1.873895,334.0
...,...,...,...,...,...,...,...,...
59,7,0,1,1,0.040439,31.727445,3.962144,229.0
60,7,1,0,0,0.020484,35.146417,1.407178,229.0
61,7,1,0,1,0.040806,32.051225,3.644348,229.0
62,7,1,1,0,0.020704,35.190196,1.659040,229.0


In [37]:
gain = (220 + 5.11)/5.11
aldo_limits = {0: {"slope": (gain*0.000445,gain*0.000485),
                    "b": (34, gain*0.86),
                    "inl": (0, 5)},
                1: {"slope": (gain*0.00089, gain*0.00096),
                    "b": (31, gain*0.77),
                    "inl": (0, 8)}}
def apply_conditions(group, aldo_limits):
    # Extract the limits for the current "gain" value from the group name
    limits = aldo_limits[group.name]
    # Apply conditions using vectorized operations
    conditions_met = (
        (group["slope"] > limits["slope"][0]) &
        (group["slope"] < limits["slope"][1]) &
        (group["b"] > limits["b"][0]) &
        (group["b"] < limits["b"][1]) &
        (group["max_inl"] > limits["inl"][0]) &
        (group["max_inl"] < limits["inl"][1])
    )
    return conditions_met

# Group the DataFrame by "gain" and apply the conditions
result = reduced_aldo.groupby("gain")[["slope", "b", "max_inl"]].apply(apply_conditions, aldo_limits=aldo_limits)

# The result is a multi-index Series; to assign it back to the original DataFrame, you might need to adjust the index
reduced_aldo["pass"] = result.reset_index(level=0, drop=True)
reduced_aldo[reduced_aldo["pass"] == False]

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
16,2,0,0,0,0.0,0.0,NaN,135.0,False
17,2,0,0,1,0.0,0.0,NaN,135.0,False
18,2,0,1,0,0.0,0.0,NaN,135.0,False
19,2,0,1,1,0.0,0.0,NaN,135.0,False
20,2,1,0,0,0.0,0.0,NaN,135.0,False
21,2,1,0,1,0.0,0.0,NaN,135.0,False
22,2,1,1,0,0.0,0.0,NaN,135.0,False
23,2,1,1,1,0.0,0.0,NaN,135.0,False


In [38]:
result

gain    
0     0     True
      2     True
      4     True
      6     True
      8     True
            ... 
1     55    True
      57    True
      59    True
      61    True
      63    True
Length: 64, dtype: bool

In [31]:
check_aldo = check_aldo(reduced_aldo)
checked_aldo

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
0,0,0,0,0,0.020495,35.412772,2.191727,334.0,True
1,0,0,0,1,0.040751,32.309174,4.052352,334.0,True
2,0,0,1,0,0.020592,35.413578,2.137131,334.0,True
3,0,0,1,1,0.041010,32.312781,4.770937,334.0,True
4,0,1,0,0,0.020424,34.736938,1.873895,334.0,True
...,...,...,...,...,...,...,...,...,...
59,7,0,1,1,0.040439,31.727445,3.962144,229.0,True
60,7,1,0,0,0.020484,35.146417,1.407178,229.0,True
61,7,1,0,1,0.040806,32.051225,3.644348,229.0,True
62,7,1,1,0,0.020704,35.190196,1.659040,229.0,True


In [32]:
checked_aldo[checked_aldo['pass'] == False]

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
16,2,0,0,0,0.0,0.0,NaN,135.0,False
17,2,0,0,1,0.0,0.0,NaN,135.0,False
18,2,0,1,0,0.0,0.0,NaN,135.0,False
19,2,0,1,1,0.0,0.0,NaN,135.0,False
20,2,1,0,0,0.0,0.0,NaN,135.0,False
21,2,1,0,1,0.0,0.0,NaN,135.0,False
22,2,1,1,0,0.0,0.0,NaN,135.0,False
23,2,1,1,1,0.0,0.0,NaN,135.0,False


In [23]:
checked_aldo[(checked_aldo["pass"]) & (checked_aldo.SN == 334)]

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
1,0,0,0,1,0.040751,32.309174,4.052352,334.0,True
3,0,0,1,1,0.041010,32.312781,4.770937,334.0,True
5,0,1,0,1,0.040636,31.665015,3.961418,334.0,True
7,0,1,1,1,0.040364,31.443128,3.860691,334.0,True


In [33]:

checked_aldo[(checked_aldo["pass"] == False) & (checked_aldo.SN == 334)]

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
0,0,0,0,0,0.020495,35.412772,2.191727,334.0,False
2,0,0,1,0,0.020592,35.413578,2.137131,334.0,False
4,0,1,0,0,0.020424,34.736938,1.873895,334.0,False
6,0,1,1,0,0.020300,34.494380,1.976460,334.0,False


In [36]:
checked_aldo[checked_aldo["pass"] == False]

,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
0,0,0,0,0,0.020495,35.412772,2.191727,334.0,False
2,0,0,1,0,0.020592,35.413578,2.137131,334.0,False
4,0,1,0,0,0.020424,34.736938,1.873895,334.0,False
6,0,1,1,0,0.020300,34.494380,1.976460,334.0,False
8,1,0,0,0,0.020349,35.077473,2.029485,290.0,False
10,1,0,1,0,0.020254,35.013717,1.998172,290.0,False
12,1,1,0,0,0.020617,35.832704,2.370462,290.0,False
14,1,1,1,0,0.020607,35.696994,2.633783,290.0,False
16,2,0,0,0,0.000000,0.000000,NaN,135.0,False
17,2,0,0,1,0.000000,0.000000,NaN,135.0,False


0     334.0
1     334.0
2     334.0
3     334.0
4     334.0
      ...  
59    229.0
60    229.0
61    229.0
62    229.0
63    229.0
Name: SN, Length: 64, dtype: float64

In [19]:
tdc = TDCCalibration(test_result_dir=str(test_result_dirs[0]), tester_to_serial=trs[0].tester_to_serial)
tdc_data = tdc.get_data()
tdc_data

,SN,chipID,branch,t0,a0,a1,a2,sigma
0,334,0,1,-0.048331,60.852036,529.64203,-0.909676,0.006707
1,334,0,2,-0.045223,55.057346,512.60339,-0.010000,0.006524
2,334,0,1,-0.032714,72.007156,531.85779,-1.601751,0.006655
3,334,0,2,-0.027873,65.236191,513.82886,-0.010001,0.006672
4,334,0,1,-0.056977,55.558521,529.12878,-0.589366,0.006768
...,...,...,...,...,...,...,...,...
8187,229,15,2,0.189833,71.365158,509.24661,-0.010000,0.006491
8188,229,15,1,0.227227,79.093002,500.37540,-1.038233,0.006523
8189,229,15,2,0.188912,71.775337,508.89395,-0.010001,0.006579
8190,229,15,1,0.227469,76.152420,499.18500,-0.812551,0.006480


In [20]:
tdc_raw_data = tdc.read_data()
tdc_raw_data

,# portID,slaveID,chipID,channelID,tacID,branch,t0,a0,a1,a2,sigma
0,0,0,0,0,0,1,-0.048331,60.852036,529.64203,-0.909676,0.006707
1,0,0,0,0,0,2,-0.045223,55.057346,512.60339,-0.010000,0.006524
2,0,0,0,0,1,1,-0.032714,72.007156,531.85779,-1.601751,0.006655
3,0,0,0,0,1,2,-0.027873,65.236191,513.82886,-0.010001,0.006672
4,0,0,0,0,2,1,-0.056977,55.558521,529.12878,-0.589366,0.006768
...,...,...,...,...,...,...,...,...,...,...,...
8187,0,0,15,31,5,2,0.189833,71.365158,509.24661,-0.010000,0.006491
8188,0,0,15,31,6,1,0.227227,79.093002,500.37540,-1.038233,0.006523
8189,0,0,15,31,6,2,0.188912,71.775337,508.89395,-0.010001,0.006579
8190,0,0,15,31,7,1,0.227469,76.152420,499.18500,-0.812551,0.006480


In [6]:
tr = TestResult(path = "/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737") 
tester_to_serial = tr.tester_to_serial
tester_to_serial

{6: 106, 0: 135, 4: 219, 2: 291, 1: 295, 5: 319, 7: 323, 3: 333}

In [7]:

pt_1000 = pd.read_csv("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737/pt1000.tsv",
                      sep="\t",
                      header=None)
rename = {0: "tester_ID", 1: "chipID", 2: "resistance"}
pt_1000 = pt_1000.rename(columns=rename)
pt_1000['SN'] = pt_1000['tester_ID'].map(tester_to_serial)
# resistance needs to be 2.25 < r < 3.75
pt_1000["pass"] = pt_1000["resistance"].apply(lambda x: 2.25 < x < 3.75)
pt_1000

,tester_ID,chipID,resistance,SN,pass
0,0,0,3.441608,135,True
1,0,1,2.683884,135,True
2,1,0,3.421166,295,True
3,1,1,2.646154,295,True
4,2,0,3.373183,291,True
5,2,1,2.670092,291,True
6,3,0,3.314595,333,True
7,3,1,2.641860,333,True
8,4,0,3.376819,219,True
9,4,1,2.635198,219,True


In [13]:
class Pt_1000(Test):
    def __init__(self,
                 name: str = "Pt_1000",
                 filename: str = "pt1000.tsv",
                 header: int | None = None,
                 index_col: int | None = None,
                 id_col: int = 0,
                 **kwargs) -> None:
        super().__init__(name=name,
                         filename=filename,
                         header=header,
                         index_col=index_col,
                         id_col=id_col, **kwargs)
    
    def get_data(self) -> pd.DataFrame:
        data = self.read_data()
        rename = {0: "tester_ID", 1: "chipID", 2: "resistance"}
        data = data.rename(columns=rename)
        data['SN'] = data['tester_ID'].map(self.tester_to_serial)
        # resistance needs to be 2.25 < r < 3.75
        data["pass"] = data["resistance"].apply(lambda x: 2.25 < x < 3.75)
        return data


pt_1000 = Pt_1000(test_result_dir=tr.path, tester_to_serial=tr.tester_to_serial)
pt_1000_data = pt_1000.get_data()
pt_1000_data

,tester_ID,chipID,resistance,SN,pass
0,0,0,3.441608,135,True
1,0,1,2.683884,135,True
2,1,0,3.421166,295,True
3,1,1,2.646154,295,True
4,2,0,3.373183,291,True
5,2,1,2.670092,291,True
6,3,0,3.314595,333,True
7,3,1,2.641860,333,True
8,4,0,3.376819,219,True
9,4,1,2.635198,219,True


In [12]:
tr.path

'/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737'

In [30]:
tec = pd.read_csv("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737/tec.tsv",
                      sep="\t",
                      header=None)
rename = {0: "tester_ID", 1: "resistance"}
tec = tec.rename(columns=rename)
tec["SN"] = tec["tester_ID"].map(tester_to_serial)
# resistance needs to be 1.1 < r < 1.5 
tec["pass"] = tec["resistance"].apply(lambda x: 1.1 < x < 1.5)
tec

,tester_ID,resistance,SN,pass
0,0,1.274510,135,True
1,1,1.316484,295,True
2,2,1.247012,291,True
3,3,1.368182,333,True
4,4,1.311741,219,True
5,5,1.364035,319,True
6,6,1.275424,106,True


In [ ]:
class Tec(Test):
    def __init__(self,
                 name: str = "Tec",
                 filename: str = "tec.tsv",
                 header: int | None = None,
                 index_col: int | None = None,
                 id_col: int = 0,
                 **kwargs) -> None:
        super().__init__(name=name,
                         filename=filename,
                         header=header,
                         index_col=index_col,
                         id_col=id_col, **kwargs)
        
    def get_data(self) -> pd.DataFrame:
        data = self.read_data()
        rename = {0: "tester_ID", 1: "resistance"}
        data = data.rename(columns=rename)
        data["SN"] = data["tester_ID"].map(self.tester_to_serial)
        # resistance needs to be 1.1 < r < 1.5 
        data["pass"] = data["resistance"].apply(lambda x: 1.1 < x < 1.5)
        return data

In [33]:
current_after_power_up = pd.read_csv("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737/current_after_power_up.tsv",
                      sep="\t",
                      header=None)
rename = {0: "tester_ID", 1: "current"}
current_after_power_up = current_after_power_up.rename(columns=rename)
# current limits : 0.02 < c < 0.2
current_after_power_up["pass"] = current.map(lambda x: 0.02 < x < 0.2)
current_after_power_up

NameError: name 'current' is not defined

In [ ]:
current_after_power_up = pd.read_csv("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737/current_after_power_up.tsv",
                      sep="\t",
                      header=None)
rename = {0: "tester_ID", 1: "current"}
current_after_power_up = current_after_power_up.rename(columns=rename)
# current limits : 0.75 < c < 0.85
current_after_power_up["pass"] = current.map(lambda x: 0.75 < x < 0.85)
current_after_power_up

In [ ]:
class CaPup(Test):
    def __init__(self,
                 name: str = "CaPup",
                 filename: str = "current_after_power_up.tsv",
                 header: int | None = None,
                 index_col: int | None = None,
                 id_col: int = 0,
                 **kwargs) -> None:
        super().__init__(name=name,
                         filename=filename,
                         header=header,
                         index_col=index_col,
                         id_col=id_col, **kwargs)
        
    def get_reduced_data(self) -> pd.DataFrame:
        data = self.read_data()
        rename = {0: "tester_ID", 1: "current"}
        data = data.rename(columns=rename)
        # current limits : 0.75 < c < 0.85
        data["pass"] = data["current"].map(lambda x: 0.75 < x < 0.85)
        return data

In [47]:
rx = pd.read_csv("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407091555/rx_phase_cmd_fail.tsv",
                 sep="\t",
                 encoding="utf-8-sig",
                 index_col=0)         
rx

,phase,m,a,count
0,0.000000,0,0,0
1,0.000000,0,1,0
2,0.000000,1,0,0
3,0.000000,1,1,0
4,0.000000,2,0,0
...,...,...,...,...
5371,0.997024,5,1,0
5372,0.997024,6,0,0
5373,0.997024,6,1,0
5374,0.997024,7,0,0
